## Load packages

In [ ]:
import networkx as nx
import random
from itertools import combinations, count
import numpy as np
import matplotlib.pyplot as plt
# create number for each group to allow use of colormap
from itertools import count, product
import osmnx as ox
import graph_funcs as gf
import os
import general_file_handling as gfh
import prepare_raster as pr

import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep
from rasterio.plot import plotting_extent
import pandas as pd

from itertools import product
from tqdm import trange
from joblib import Parallel, delayed
from tqdm.notebook import trange, tqdm

from itertools import product

import pickle    
from matplotlib.lines import Line2D
import matplotlib as mpl
import seaborn as sns


c:\Users\hanna\Anaconda3\envs\geothings_ox\Lib\site-packages\osmnx\projection.py:3: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd
c:\Users\hanna\Anaconda3\envs\geothings_ox\Lib\site-packages\libpysal\cg\alpha_shapes.py:38: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator

In [ ]:
from importlib.metadata import version
version('osmnx')
from collections import Counter, defaultdict

pathname = '../Data/dem2/ASTGTMV003_N08W074'
from rasterio.plot import show
from rasterio.merge import merge
import rasterio as rio
from pathlib import Path
import glob
from shapely.geometry import Point, LineString


## Merge raster


In [ ]:
path = 'C:/Users/hanna/Documents/EPA/scriptie/shp/*.tif'
# Path('output').mkdir(parents=True, exist_ok=True)
output_path = 'output/mosaic_output.tif'
if os.path.exists(output_path):
    print('file exists')
else:

    raster_files =[]
    raster_files.extend(glob.glob(path))

    raster_to_mosiac = []
    for p in raster_files:
        raster = rio.open(p)
        raster_to_mosiac.append(raster)

    mosaic, output = merge(raster_to_mosiac)

    output_meta = raster.meta.copy()
    output_meta.update(
        {"driver": "GTiff",
            "height": mosaic.shape[1],
            "width": mosaic.shape[2],
            "transform": output,
        })

    with rio.open(output_path, "w", **output_meta) as m:
        m.write(mosaic)

file exists


## edge attributes

In [ ]:
def add_elev(G_proj, pathname):
    # G_proj = ox.project_graph(G_proj, to_crs=4326)
    # (print(G_proj.nodes))
    G_proj = ox.add_node_elevations_raster(G_proj,pathname )
    return G_proj

In [ ]:

#load graph
graph_counter_l_prev = 10
G_wrl_cnct = ox.io.load_graphml(filepath=pathname+f'_graph_{graph_counter_l_prev}_l.osm')



# G = ox.add_edge_travel_times(ox.add_edge_speeds(G))


In [ ]:
nodes_gtt, edges_gtt= ox.utils_graph.graph_to_gdfs(G_wrl_cnct)

In [ ]:

def reset_geom(row):

    x_start = nodes_gtt.loc[row['u']]['x']
    y_start = nodes_gtt.loc[row['u']]['y']
    x_end = nodes_gtt.loc[row['v']]['x']
    y_end = nodes_gtt.loc[row['v']]['y']
    
    row['geometry_old_pnt'] = LineString([row['geometry'].coords[0], row['geometry'].coords[-1]])


    row['geometry'] = LineString([[x_start,y_start],[x_end,y_end]])
    
    return row


In [ ]:
#TODO: een verbinding met twee nodes, maar wel een enorme afstand
G_tt_reset = edges_gtt.reset_index()


G_tt_reset_old= G_tt_reset.to_crs(21897)

G_tt_reset['geometry_old_proj'] = G_tt_reset_old['geometry']
G_tt_reset['geometry_old'] = G_tt_reset['geometry']



# G_tt_reset = G_tt_reset.to_crs(21897)
# G_tt_reset['length_old'] = [row['geometry'].length for i,row in G_tt_reset.iterrows()]
# G_tt_reset = G_tt_reset.to_crs(4326)
# G_tt_reset.loc[G_tt_reset['type'] == 'air', 'length'] = G_tt_reset.loc[G_tt_reset['type'] == 'air', 'length_old']

G_tt_reset= G_tt_reset.apply(reset_geom, axis=1)
G_tt_reset['new_geom'] = G_tt_reset['geometry']
G_tt_reset_proj = G_tt_reset.to_crs(21897)
G_tt_reset_proj['length_old'] = [row['geometry_old_proj'].length for i,row in G_tt_reset_proj.iterrows()]

#recalculate length
G_tt_reset_proj['length_new'] = [row['geometry'].length for i,row in G_tt_reset_proj.iterrows()]

G_tt_reset_proj['length_diff'] = G_tt_reset_proj['length_new'] - G_tt_reset_proj['length_old'] 
G_tt_reset_proj = G_tt_reset_proj.to_crs(4326)


#remove too long water road edges
#also select water edges that only have two points

In [ ]:
G_tt_reset_proj_save = G_tt_reset_proj

In [ ]:
G_tt_reset_proj_wr = G_tt_reset_proj.loc[((G_tt_reset_proj['type'] == 'water') | (G_tt_reset_proj['type'] == 'water_road') | (G_tt_reset_proj['type'] == 'road'))]
G_tt_reset_proj_wr_sorted = G_tt_reset_proj_wr.loc[G_tt_reset_proj['length_diff'] >500]



In [ ]:
#TODO: create list with counts of both u and v
from collections import Counter
u_list = G_tt_reset_proj_wr_sorted['u'].to_list()
v_list = G_tt_reset_proj_wr_sorted['v'].to_list()
uv_list = u_list+v_list

counts= Counter(uv_list)


In [ ]:
nodes_lst = []
for key,count in counts.items():  # for name, age in dictionary.iteritems():  (for Python 2.x)
    if count >1 :
        nodes_lst.append(key)

In [ ]:
nodes_lst_dct = dict.fromkeys((64654327, 64381833, 64133978, 63590120, 64627310, 64584751, 64617184, 64637139, 64681666, 63875838, 61030720, 65091159))

count=0
for i,row in G_tt_reset_proj_wr_sorted.sort_values(by='v').iterrows():
    if row['u'] in nodes_lst:
        # print(row)
        nodes_lst_dct[row['u']] = [row['geometry_old'].coords[0], count]
        count+=1

In [ ]:
for i, row in G_tt_reset_proj_wr_sorted.iterrows():
    if row['u'] in nodes_lst_dct.keys():
        # print(G_tt_reset_proj.loc[(G_tt_reset_proj['u'] == row['u']) &(G_tt_reset_proj['v'] == row['v'])&(G_tt_reset_proj['key'] == row['key'])])
        G_tt_reset_proj.loc[(G_tt_reset_proj['u'] == row['u']) &(G_tt_reset_proj['v'] == row['v'])&(G_tt_reset_proj['key'] == row['key']), 'u'] = nodes_lst_dct[row['u']][1]
    if row['v'] in nodes_lst_dct.keys():
        G_tt_reset_proj.loc[(G_tt_reset_proj['u'] == row['u']) &(G_tt_reset_proj['v'] == row['v'])&(G_tt_reset_proj['key'] == row['key']), 'v'] = nodes_lst_dct[row['v']][1]

In [ ]:
nodes_gtt.reset_index(inplace=True)

In [ ]:

for key in nodes_lst_dct.keys():
    x = nodes_lst_dct[key][0][0]
    y = nodes_lst_dct[key][0][1]
    geom = Point(x,y)
    df = pd.DataFrame.from_dict({'osmid': [nodes_lst_dct[key][1]], 'x':[x], 'y': [y], 'geometry': [geom] })
    nodes_gtt = pd.concat([nodes_gtt, df], ignore_index=True)


In [ ]:
G_tt_reset_proj['geometry'] = G_tt_reset_proj['geometry_old']
G_tt_reset_proj = G_tt_reset_proj.set_index(['u', 'v', 'key'])


In [ ]:
nodes_gtt = nodes_gtt.set_index("osmid")
nodes_gtt = nodes_gtt.set_geometry("geometry")

In [ ]:
G_wrl_cnct_fix =  ox.utils_graph.graph_from_gdfs(nodes_gtt, G_tt_reset_proj, {'crs': 'EPSG:4326'})


In [ ]:
#add elevation
G_elev = add_elev(G_wrl_cnct_fix, output_path)

#add edge grades
G_grade = ox.elevation.add_edge_grades(G_elev)

G_speed = ox.add_edge_speeds(G_grade)


c:\Users\hanna\Anaconda3\envs\geothings_ox\Lib\site-packages\osmnx\elevation.py:68: RuntimeWarning: divide by zero encountered in divide
  grades = ((elevs[:, 1] - elevs[:, 0]) / np.array(lengths)).round(precision)
c:\Users\hanna\Anaconda3\envs\geothings_ox\Lib\site-packages\osmnx\elevation.py:68: RuntimeWarning: invalid value encountered in divide
  grades = ((elevs[:, 1] - elevs[:, 0]) / np.array(lengths)).round(precision)


In [ ]:
#make gdf from graph
nodes_g, edges_g = ox.utils_graph.graph_to_gdfs(G_speed)

In [ ]:
edges_g['modality'] = None
edges_g['fuel_consump'] = None
edges_g['monthly_wage'] = None
edges_g['travel_risk'] = None
edges_g['transit_cost'] = None
edges_g['transit_risk'] = None
edges_g['load'] = None

In [ ]:
# Assign Small ports = VEPBL COBAQ

edges_g.loc[(63816700, 63816707,0),'type'] = 'port_connect_small'
edges_g.loc[(62975570, 62972555, 0),'type'] = 'port_connect_small'
nodes_g.loc[63816700,'type'] = 'port_small'
nodes_g.loc[62975570,'type'] = 'port_small'


C:\Users\hanna\AppData\Local\Temp\ipykernel_4744\1941580811.py:3: PerformanceWarning: indexing past lexsort depth may impact performance.
  edges_g.loc[(63816700, 63816707,0),'type'] = 'port_connect_small'
C:\Users\hanna\AppData\Local\Temp\ipykernel_4744\1941580811.py:4: PerformanceWarning: indexing past lexsort depth may impact performance.
  edges_g.loc[(62975570, 62972555, 0),'type'] = 'port_connect_small'


In [ ]:
with open("./ports_distances.csv", 'rb') as f:  # notice the r instead of w
    port_dist = pd.read_csv(f, delimiter= ';')


In [ ]:
port_dist

,Port_1,Port_2,Dist_km,Time (uur),Speed (knots),Modality,osmid
0,COCTG,ESALG,7597.420,293.0,25.928,Vessel,"[30704232, 65132273]"
1,FRDKK,ESALG,2309.110,89.0,25.928,Vessel,"[30704232, 53195375]"
2,NLRTM,ESALG,2506.070,96.0,25.928,Vessel,"[30704232, 56020150]"
3,ESVLC,BEANR,3213.840,123.0,25.928,Vessel,"[55656241, 39117447]"
4,PAMIT,BEANR,9517.870,367.0,25.928,Vessel,"[55656241, 70520231]"
...,...,...,...,...,...,...,...
90,VEPBL,NLRTM,7924.708,306.0,25.928,Vessel,"[56020150, 57526123]"
91,VELAG,NLVLI,7758.028,299.0,25.928,Vessel,"[55039437, 56350873]"
92,VEPBL,NLVLI,7841.368,302.0,25.928,Vessel,"[55039437, 57526123]"
93,VELAG,VEGUT,268.540,10.0,25.928,Feeder,"[54342630, 56350873]"


In [ ]:
def label_modality(row):
     #TODO: check port connect
     label_lst = ['water_road_overside', 'road', 'water_road',
          'field_connect', 'airstrip_connect', 'lab_connect', 'port_connect', 'water_road_overside']
     vessel_lst= port_dist.loc[port_dist['Modality'] == 'Vessel'].osmid.to_list()
     sea_edges= port_dist.osmid.to_list()
     
     if row['type'] in label_lst and row['grade'] < 30:
          row['modality'] = 'truck'
          row['fuel_consump'] = 0.11
          row['monthly_wage'] = 1020
          row['travel_risk'] = 0.02
          row['load'] = 100
          if np.isnan(row['speed_kph']):
               # print(row)
               row['speed_kph'] = 41.265345
     elif row['type'] in label_lst:
          row['modality'] =  'donkey'
          row['speed_kph'] = 5
          row['fuel_consump'] = 0
          row['monthly_wage'] = 1020
          row['travel_risk'] = 0.01
          row['load'] = 50

     elif row['type'] == 'water':
          row['modality'] =  'motorboat'
          row['speed_kph'] = 45
          row['fuel_consump'] = 0.22
          row['monthly_wage'] = 1020
          row['travel_risk'] = 0.02
          row['load'] = 200

     elif row['type'] == 'air':
          row['modality'] =  'plane'
          row['speed_kph'] = 357
          row['fuel_consump'] = 0.24
          row['monthly_wage'] = 1700
          row['travel_risk'] = 0.1
          row['load'] = 500
          if row['length'] > 1700000:
               row['type'] = 'air_remove'

     elif row['type'] == 'sea' and str(row['osmid']) in vessel_lst:
          row['modality'] =  'vessel'
          row['speed_kph'] = 23.73
          # print(row['osmid'])
          row['length'] = port_dist.loc[port_dist['osmid'] == str(row['osmid'])].Dist_km.to_list()[0]*1000
          row['fuel_consump'] = 0
          row['monthly_wage'] = 0
          row['travel_risk'] = 0.05
          row['load'] = 500

     elif row['type'] == 'sea':
          row['modality'] =  'feeder'
          row['speed_kph'] = 23.73
          # print(row['osmid'])
          row['length'] = port_dist.loc[port_dist['osmid'] == str(row['osmid'])].Dist_km.to_list()[0]*1000
          row['fuel_consump'] = 0
          row['monthly_wage'] = 0
          row['travel_risk'] = 0.05
          row['load'] = 500

     else:
          #TODO: check if this is correct/makes sense?
          row['speed_kph'] = 0
          
     if row['length'] == 0:
          row['modality'] = 'no modality'

     return row

In [ ]:
edges_g_mods = edges_g.apply(label_modality, axis=1)

In [ ]:

# convert distance meters to km, and speed km per hour to km per second
distance_km = edges_g_mods["length"] / 1000
speed_km_sec = edges_g_mods["speed_kph"] / (60 * 60)

# calculate edge travel time in seconds
travel_time = distance_km / speed_km_sec

# add travel time attribute to graph edges
edges_g_mods["travel_time"] = travel_time.round(1).to_numpy()


Path Type                  Transit Time (hours)
- Water road connection          0.5
- Small port path                12
- Large port path                12
- Airstrip path                  0.75
- Seaway                         24
- Airway                         1.5


'road''air''water''water_road' 'lab_connect''field_connect''water_road_overside''transhipment_airstrip''airstrip_connect''sea' 'transhipment_port' 'port_connect'


In [ ]:
def add_transit_times(row):
     #TODO: check port connect
     # print(row['type'])
    
     if row['type'] == 'water_road' or row['type'] == 'water_road_overside':
          row['travel_time'] +=(0.5*60*60)
          row['transit_cost'] = 25
          row['transit_risk'] = 0.3
          
     elif row['type'] == 'airstrip_connect':
          row['travel_time'] +=(0.75*60*60)
          row['transit_cost'] = 25
          row['transit_risk'] = 0.15

     elif row['type'] == 'air':
          row['travel_time'] +=(1.5*60*60)
          row['transit_cost'] = 50
          row['transit_risk'] = 0.5

     elif row['type'] == 'sea':
          # print(row['travel_time'])
          row['travel_time'] +=(24*60*60)
          row['transit_cost'] = 50
          row['transit_risk'] = 0.5

          # print(row['travel_time'])
     elif row['type'] == 'port_connect_small':
          # print(row['travel_time'])
          row['travel_time'] +=(12*60*60)
          row['transit_cost'] = 25
          row['transit_risk'] = 0.25

     elif row['type'] == 'port_connect':
          # print(row['travel_time'])
          row['travel_time'] +=(12*60*60)
          row['transit_cost'] = 50
          row['transit_risk'] = 0.55
     else:
          row['transit_cost'] = 0
          row['transit_risk'] = 0


          # print(row['travel_time'])
          
     return row

In [ ]:
edges_gtt = edges_g_mods.apply(add_transit_times, axis=1)

In [ ]:
def add_total_cost(row):
    km_wage = row['monthly_wage']/((42*52)/12)
    if row['speed_kph'] == 0:
        wage=0
    else:
        wage = (km_wage/row['speed_kph'])
    travel_cost = wage*row['length'] + row['fuel_consump']*row['length']
    transit_cost = row['load']* row['transit_cost']
    row['cost'] = (transit_cost+travel_cost)*(500/row['load'])
    return row

def add_total_risk(row):
    row['risk'] = row['transit_risk'] + row['travel_risk']
    return row



In [ ]:
edges_g['cost'] = None
edges_gcost = edges_gtt.apply(add_total_cost, axis=1)

edges_g['risk'] = None
edges_gtrc = edges_gcost.apply(add_total_risk, axis=1)


In [ ]:
G_tt= ox.utils_graph.graph_from_gdfs(nodes_g, edges_gtrc, {'crs': 'EPSG:4326'})
# G_tt = ox.add_edge_travel_times(G_comb)
# #make gdf from graph
# nodes_gtt, edges_gtt = ox.utils_graph.graph_to_gdfs(G_tt)

In [ ]:
ox.io.save_graphml(G_tt, filepath=pathname+'_graph_G_final.osm', encoding='utf-8')
